In [ ]:
(define-syntax-rule (lazy body body* ...)
  (let ([forced #f]
	[value #f])
    (lambda ()
      (unless forced
	(set! value (begin body body* ...))
	(set! forced #t))
      value)))

(define (force lazy-value)
  (lazy-value))

(define-syntax-rule (lazy-cons head tail)
  (lazy (cons head tail)))

(define (lazy-car lst)
  (car (force lst)))

(define (lazy-cdr lst)
  (cdr (force lst)))

(define (lazy-nil)
  (lazy '()))

(define (lazy-null? x)
  (null? (force x)))

(define (make-lazy lst)
  (lazy (if (null? lst)
	    '()
	    (cons (car lst) (make-lazy (cdr lst))))))

(define (take n lst)
  (if (or (<= n 0) (lazy-null? lst))
      '()
      (cons (lazy-car lst) (take (1- n) (lazy-cdr lst)))))

(define (take-all lst)
  (if (lazy-null? lst)
      '()
      (cons (lazy-car lst) (take-all (lazy-cdr lst)))))

(define (lazy-map f lst)
  (lazy (if (lazy-null? lst)
	    '()
	    (cons (f (lazy-car lst))
		  (lazy-map f (lazy-cdr lst))))))

(define (lazy-mapcat f lst)
  (letrec ([helper (lambda (lst-cur)
		     (if (lazy-null? lst-cur)
			 (force (lazy-mapcat f (lazy-cdr lst)))
			 (cons (lazy-car lst-cur)
			       (lazy (helper (lazy-cdr lst-cur))))))])
    (lazy (if (lazy-null? lst)
	      '()
	      (helper (f (lazy-car lst)))))))

(define (lazy-find-if pred? lst)
  (if (lazy-null? lst)
      '()
      (let ([head (lazy-car lst)])
	(if (pred? head)
	    head
	    (lazy-find-if pred? (lazy-cdr lst))))))

(define (lazy-nth n lst)
  (if (zero? n)
      (lazy-car lst)
      (lazy-nth (1- n) (lazy-cdr lst))))

In [ ]:
(define integers
  (letrec ([f (lambda (n)
		(lazy-cons n (f (1+ n))))])
	  (f 1)))

(define evens (lazy-mapcat (lambda (x)
			     (if (even? x)
				 (make-lazy (list x))
				 (lazy-nil)))
			   integers))

In [ ]:
(take 20 evens)

In [ ]:
(lazy-find-if (lambda (x)
                (> x 11))
              evens)

In [ ]:
(lazy-nth 100 evens)